In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Predicting the Spread of Covid-19 with Linear Regression

The array `num_cases` contains the number of cases on successive days in the time period from February, the 25th up to the 13th of March 2020, i.e. `num_cases[0]` is the number of Covid-19 cases on the 25th of February and `num_cases[i]`is the number of Covid-19 cases `i` days after the 25th of February.  I have taken these data form the website <a href="http://interaktiv.morgenpost.de/corona-virus-karte-infektionen-deutschland-weltweit/">http://interaktiv.morgenpost.de/corona-virus-karte-infektionen-deutschland-weltweit/</a>.

In [ ]:
num_cases = [16, 19, 24, 53, 66, 117, 150, 188, 240, 349, 534, 684, 847, 1112, 1565, 1966, 2745, 3675]

`m` is the number of data points.

In [ ]:
m = len(num_cases)
m

We take the number of days after the 25th of February as x-values.

In [ ]:
X = range(m)

In [ ]:
list(X)

To begin with, we plot the data.  To this end we have to load some libraries.

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

In [ ]:
plt.figure(figsize=(12, 10))
sns.set(style='darkgrid')
plt.xticks(ticks=X)
plt.scatter(X, num_cases, c='r') 
plt.xlabel('date in number of days since the 25th of Febuary')
plt.ylabel('number of Covid-19 cases')
plt.title('Number of Covid-19 cases versus date.')

This does not look linear.  Rather, it might be exponential.  Hence, we take the logarithm of the number of cases and plot these logarithms with respect to the date.

In [ ]:
import math

In [ ]:
Y = [math.log(y) for y in num_cases]

In [ ]:
plt.figure(figsize=(12, 10))
sns.set(style='whitegrid')
plt.xticks(ticks=X)
plt.scatter(X, Y, c='r') 
plt.xlabel('date in number of days since the 25th of Febuary')
plt.ylabel('log of the number of Covid-19 cases')
plt.title('Log of the Number of Covid-19 cases versus date.')

This looks better. There seem to be a jump at `x = 3` and `x = 5`. These correspond to the date of Friday, the 28th of February and Sunday, the 1st of March.  These dates coincide with the German *Carnival* in 2020.  This period might have enhanced the normal spreading of the disease.  Therefore, let us just take the data starting from the 1st of March.

To ease the computation, we transform our data into numpy vectors.  Note that we throw away the first 5 data.

In [ ]:
X = np.array(X[5:])
Y = np.array(Y[5:])

In [ ]:
X

We compute the average value of `X` according to the formula:
$$ \bar{\mathbf{x}} = \frac{1}{m} \cdot \sum\limits_{i=1}^m x_i $$ 

In [ ]:
xMean = np.mean(X)
xMean

We compute the average number of the logarithm of Covid-19 cases according to the formula:
$$ \bar{\mathbf{y}} = \frac{1}{m} \cdot \sum\limits_{i=1}^m y_i $$ 

In [ ]:
yMean = np.mean(Y)
yMean

The coefficient $\vartheta_1$ is computed according to the formula:
$$ \vartheta_1 = \frac{\sum\limits_{i=1}^m \bigl(x_i - \bar{\mathbf{x}}\bigr) \cdot \bigl(y_i - \bar{\mathbf{y}}\bigr)}{
                       \sum\limits_{i=1}^m \bigl(x_i - \bar{\mathbf{x}}\bigr)^2}  
$$

In [ ]:
ϑ1 = np.sum((X - xMean) * (Y - yMean)) / np.sum((X - xMean) ** 2)
ϑ1

The coefficient $\vartheta_0$ is computed according to the formula:
$$ \vartheta_0 = \bar{\mathbf{y}} - \vartheta_1 \cdot \bar{\mathbf{x}} $$ 

In [ ]:
ϑ0 = yMean - ϑ1 * xMean
ϑ0

Let us plot the line $y(x) = ϑ0 + ϑ1 \cdot x$ together with our data:

In [ ]:
xMax = max(X) + 0.2
plt.figure(figsize=(12, 10))
sns.set(style='darkgrid')
plt.xticks(ticks=X)
plt.scatter(X, Y, c='r')
plt.plot([0, xMax], [ϑ0, ϑ0 + ϑ1 * xMax], c='b')
plt.xlabel('date in number of days since the 1st of March')
plt.ylabel('log of the number of Covid-19 cases')
plt.title('Log of the Number of Covid-19 cases versus date.')

The blue line is not too far of the data points.  In order to judge the quality of our model we compute both the
<em style="color:blue">total sum of squares</em> and the <em style="color:blue">residual sum of squares</em>.

In [ ]:
TSS = np.sum((Y - yMean) ** 2)

Next, we compute the *residual sum of squares* `RSS` as follows:
$$ \mathtt{RSS} := \sum\limits_{i=1}^m \bigl(\vartheta_1 \cdot x_i + \vartheta_0 - y_i\bigr)^2 $$
    

In [ ]:
RSS = np.sum((ϑ1 * X + ϑ0 - Y) ** 2)

Now $R^2$ is calculated via the formula:
$$ R^2 = 1 - \frac{\mathtt{RSS}}{\mathtt{TSS}}$$

In [ ]:
R2 = 1 - RSS/TSS
R2

It seems that our model is a good approximation of the data.

Finally, let us make a prediction for the 16th of March. The logarithm of the number of cases $n$ on the 16th of March which is $20$ days after the 25th of February is predicted according to the formula:
$$ \ln(n) = \vartheta_1 \cdot 20 + \vartheta_0 $$
Hence, the number of cases $n$ for that day is predicted as
$$ n = \exp\bigl(\vartheta_1 \cdot 20 + \vartheta_0\bigr). $$

In [ ]:
round(math.exp(ϑ1 * 20 + ϑ0))

Our model predicts that there are about $8,787$ cases by the end of Monday, the 16th of March.  
(The actual number of cases was $8,919$.)
